In [72]:
import pandas as pd

In [73]:
import numpy as np

### Carregar dados

In [74]:
jogos = pd.read_csv("data/brasileiro-a.csv")

#### A query abaixo possibilita restringir o espaço amostral

In [75]:
#jogos.query('ano = 2017 & rodada > 25', inplace=True)

In [76]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,placar
0,brasileiro-a,2017,30,23.10. 19:00,Botafogo,Corinthians,2 : 1
1,brasileiro-a,2017,30,22.10. 18:00,Chapecoense,Fluminense,2 : 0
2,brasileiro-a,2017,30,22.10. 18:00,Ponte Preta,Avaí,1 : 2
3,brasileiro-a,2017,30,22.10. 16:00,Atlético-PR,Sport Recife,2 : 1
4,brasileiro-a,2017,30,22.10. 16:00,Bahia,Vitória,2 : 1


In [77]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3340 entries, 0 to 3339
Data columns (total 7 columns):
campeonato    3340 non-null object
ano           3340 non-null int64
rodada        3340 non-null int64
data          3340 non-null object
mandante      3340 non-null object
visitante     3340 non-null object
placar        3340 non-null object
dtypes: int64(2), object(5)
memory usage: 182.7+ KB


In [79]:
jogos_obj = jogos.select_dtypes(['object'])
jogos[jogos_obj.columns] = jogos_obj.apply(lambda x: x.str.strip())

### Lista de times participantes

In [80]:
times = list(jogos["mandante"].unique())

In [81]:
times.sort()

In [82]:
times

['América-MG',
 'Atlético-GO',
 'Atlético-MG',
 'Atlético-PR',
 'Avaí',
 'Bahia',
 'Barueri',
 'Botafogo',
 'Ceará',
 'Chapecoense',
 'Corinthians',
 'Coritiba',
 'Criciúma',
 'Cruzeiro',
 'Figueirense',
 'Flamengo',
 'Fluminense',
 'Goias',
 'Grêmio',
 'Guarani',
 'Internacional',
 'Joinville',
 'Náutico',
 'Palmeiras',
 'Ponte Preta',
 'Portuguesa',
 'Santa Cruz',
 'Santo Andre',
 'Santos',
 'Sport Recife',
 'São Paulo',
 'Vasco',
 'Vitória']

In [83]:
len(times)

33

### Transformar placar em 2 colunas

In [84]:
def tratarPlacar(row, mandante):
    placar = row["placar"].split(" : ")
    if (mandante):
        return int(placar[0])
    return int(placar[1])

In [85]:
jogos['gols_mandante'] = jogos.apply (lambda row: tratarPlacar(row, True),axis=1)

In [86]:
jogos['gols_visitante'] = jogos.apply (lambda row: tratarPlacar(row, False),axis=1)

In [87]:
del jogos["placar"]

In [89]:
jogos.describe()

,ano,rodada,gols_mandante,gols_visitante
count,3340.000000,3340.000000,3340.000000,3340.000000
mean,2012.904192,19.140719,1.504192,0.994910
std,2.536900,10.840727,1.196348,0.993075
min,2009.000000,1.000000,0.000000,0.000000
25%,2011.000000,10.000000,1.000000,0.000000
50%,2013.000000,19.000000,1.000000,1.000000
75%,2015.000000,28.000000,2.000000,2.000000
max,2017.000000,38.000000,6.000000,6.000000


### Tratar coluna de data

In [90]:
def tratarData(row):
    data = row["data"].split(".")    
    return data[1] + "-" + data[0]

In [91]:
jogos['data'] = jogos.apply (lambda row: tratarData(row),axis=1)

In [92]:
jogos.head(2)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante
0,brasileiro-a,2017,30,10-23,Botafogo,Corinthians,2,1
1,brasileiro-a,2017,30,10-22,Chapecoense,Fluminense,2,0


### Criar coluna com resultado

In [93]:
def resultado(row):
    if (row["gols_mandante"] > row["gols_visitante"]):
        return "MANDANTE"
    elif (row["gols_mandante"] < row["gols_visitante"]):
        return "VISITANTE"
    else:
        return "EMPATE"

In [94]:
jogos['resultado'] = jogos.apply (lambda row: resultado(row),axis=1)

In [95]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado
0,brasileiro-a,2017,30,10-23,Botafogo,Corinthians,2,1,MANDANTE
1,brasileiro-a,2017,30,10-22,Chapecoense,Fluminense,2,0,MANDANTE
2,brasileiro-a,2017,30,10-22,Ponte Preta,Avaí,1,2,VISITANTE
3,brasileiro-a,2017,30,10-22,Atlético-PR,Sport Recife,2,1,MANDANTE
4,brasileiro-a,2017,30,10-22,Bahia,Vitória,2,1,MANDANTE


### Totais de resultados por ano

In [96]:
jogos.groupby(["ano", "resultado"])[["ano"]].count()

ano
ano  resultado     
2009 EMPATE     102
     MANDANTE   195
     VISITANTE   83
2010 EMPATE     118
     MANDANTE   179
     VISITANTE   83
2011 EMPATE     105
     MANDANTE   184
     VISITANTE   91
2012 EMPATE     105
     MANDANTE   183
     VISITANTE   92
2013 EMPATE     108
     MANDANTE   184
     VISITANTE   88
2014 EMPATE      92
     MANDANTE   197
     VISITANTE   91
2015 EMPATE      91
     MANDANTE   200
     VISITANTE   89
2016 EMPATE      95
     MANDANTE   202
     VISITANTE   83
2017 EMPATE      77
     MANDANTE   130
     VISITANTE   93

### Totais por resultado

In [97]:
totais_resultados = jogos.groupby("resultado").size()

In [98]:
total_jogos = totais_resultados.sum()

In [99]:
resultados_DF = pd.DataFrame({'resultados': totais_resultados.keys(), 'totais': totais_resultados.values})

In [100]:
resultados_DF

,resultados,totais
0,EMPATE,893
1,MANDANTE,1654
2,VISITANTE,793


In [101]:
def percentual(valor, total):
    return round(valor/total, 4)

In [102]:
resultados_DF['%'] = resultados_DF.apply (lambda row: percentual(row["totais"], total_jogos),axis=1)

In [103]:
resultados_DF

,resultados,totais,%
0,EMPATE,893,0.2674
1,MANDANTE,1654,0.4952
2,VISITANTE,793,0.2374


### Calcular resultados de um time

In [104]:
def resultadosTime(time, casa, resultado):
    return len(jogos[(jogos[casa] == time) & (jogos["resultado"] == resultado)]) 

In [105]:
resultados_time_DF = pd.DataFrame({'time': times })

In [106]:
resultados_time_DF["vitorias_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "MANDANTE"),axis=1)
resultados_time_DF["empates_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "VISITANTE"),axis=1)
resultados_time_DF["vitorias_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "VISITANTE"),axis=1)
resultados_time_DF["empates_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "MANDANTE"),axis=1)

In [107]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora
0,América-MG,14,10,14,1,10,27
1,Atlético-GO,24,15,33,13,20,39
2,Atlético-MG,97,32,38,46,49,72


In [108]:
resultados_time_DF.eval("jogos_casa = vitorias_casa + empates_casa + derrotas_casa", inplace=True)
resultados_time_DF.eval("jogos_fora = vitorias_fora + empates_fora + derrotas_fora", inplace=True)
resultados_time_DF.eval("jogos_total = jogos_fora + jogos_fora", inplace=True)

In [109]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total
0,América-MG,14,10,14,1,10,27,38,38,76
1,Atlético-GO,24,15,33,13,20,39,72,72,144
2,Atlético-MG,97,32,38,46,49,72,167,167,334


In [110]:
def aproveitamentoMandante(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    return (3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante)

In [111]:
resultados_time_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: aproveitamentoMandante(row),axis=1)
resultados_time_DF.head(3)


,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa
0,América-MG,14,10,14,1,10,27,38,38,76,0.456140
1,Atlético-GO,24,15,33,13,20,39,72,72,144,0.402778
2,Atlético-MG,97,32,38,46,49,72,167,167,334,0.644711


In [112]:
def aproveitamentoVisitante(row):
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"]) / (3*total_jogos_visitante)

In [113]:
resultados_time_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: aproveitamentoVisitante(row),axis=1)
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora
0,América-MG,14,10,14,1,10,27,38,38,76,0.456140,0.114035
1,Atlético-GO,24,15,33,13,20,39,72,72,144,0.402778,0.273148
2,Atlético-MG,97,32,38,46,49,72,167,167,334,0.644711,0.373253


In [114]:
def aproveitamentoTotal(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"] + 3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante + 3*total_jogos_visitante)

In [115]:
resultados_time_DF["aprov_total"] = resultados_time_DF.apply (lambda row: aproveitamentoTotal(row),axis=1)
resultados_time_DF.sort_values(["aprov_total"], ascending=[0]).head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total
10,Corinthians,103,40,24,52,57,58,167,167,334,0.696607,0.425150,0.560878
13,Cruzeiro,90,42,35,63,34,70,167,167,334,0.622754,0.445110,0.533932
18,Grêmio,104,36,27,45,51,71,167,167,334,0.694611,0.371257,0.532934


In [116]:
resultados_time_DF = resultados_time_DF[["time", "jogos_casa", "jogos_fora", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total"]].sort_values(["aprov_total"], ascending=[0])

In [117]:
def golsMandante(time):
    print(time)
    return jogos[jogos["mandante"] == time][["gols_mandante"]].sum()

In [118]:
def golsVisitante(time):
    return jogos[jogos["visitante"] == time]["gols_visitante"].sum()

In [119]:
resultados_time_DF["gols_mandante"] = resultados_time_DF.apply (lambda row: golsMandante(row["time"]),axis=1)
resultados_time_DF["gols_visitante"] = resultados_time_DF.apply (lambda row: golsVisitante(row["time"]),axis=1)

resultados_time_DF.eval("total_gols = gols_mandante + gols_visitante", inplace=True)
resultados_time_DF.eval("media_gols_casa = gols_mandante/jogos_casa", inplace=True)
resultados_time_DF.eval("media_gols_fora = gols_visitante/jogos_fora", inplace=True)
resultados_time_DF.eval("media_gols = (gols_mandante + gols_visitante)/jogos_total", inplace=True)

Corinthians
Corinthians
Cruzeiro
Grêmio
São Paulo
Atlético-MG
Santos
Internacional
Fluminense
Flamengo
Palmeiras
Botafogo
Atlético-PR
Vasco
Chapecoense
Coritiba
Portuguesa
Goias
Ponte Preta
Sport Recife
Vitória
Bahia
Avaí
Figueirense
Ceará
Santo Andre
Barueri
Criciúma
Atlético-GO
Guarani
Náutico
América-MG
Joinville
Santa Cruz


In [120]:
resultados_time_DF.head()


,time,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total,gols_mandante,gols_visitante,total_gols,media_gols_casa,media_gols_fora,media_gols
10,Corinthians,167,167,334,0.696607,0.425150,0.560878,276,178,454,1.652695,1.065868,1.359281
13,Cruzeiro,167,167,334,0.622754,0.445110,0.533932,281,197,478,1.682635,1.179641,1.431138
18,Grêmio,167,167,334,0.694611,0.371257,0.532934,282,172,454,1.688623,1.029940,1.359281
30,São Paulo,167,167,334,0.646707,0.383234,0.514970,276,185,461,1.652695,1.107784,1.380240
2,Atlético-MG,167,167,334,0.644711,0.373253,0.508982,293,190,483,1.754491,1.137725,1.446108


In [121]:
resultados_time_DF = resultados_time_DF[["time", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total", "media_gols_casa", "media_gols_fora", "media_gols"]]

In [122]:
resultados_time_DF.set_index("time", inplace=True)

In [123]:
resultados_time_DF.sort_values(["aprov_total"], ascending = [0]).head(3)

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
time,,,,,,,
Corinthians,334,0.696607,0.425150,0.560878,1.652695,1.065868,1.359281
Cruzeiro,334,0.622754,0.445110,0.533932,1.682635,1.179641,1.431138
Grêmio,334,0.694611,0.371257,0.532934,1.688623,1.029940,1.359281


In [124]:
resultados_time_DF_DESC = resultados_time_DF.describe(percentiles=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

In [125]:
resultados_time_DF_DESC

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,202.424242,0.549986,0.283632,0.416809,1.426494,0.939307,1.182900
std,110.259645,0.080022,0.093634,0.080337,0.213646,0.154631,0.170976
min,38.000000,0.402778,0.105263,0.271930,0.894737,0.368421,0.684211
0%,38.000000,0.402778,0.105263,0.271930,0.894737,0.368421,0.684211
10%,45.600000,0.458947,0.141520,0.315205,1.154737,0.797895,0.991579
20%,76.000000,0.476023,0.210526,0.345614,1.220760,0.842105,1.040659
30%,132.000000,0.501818,0.236577,0.377193,1.321558,0.875676,1.130556
40%,182.000000,0.524912,0.269481,0.400333,1.421053,0.914765,1.167345
50%,190.000000,0.551515,0.286036,0.407018,1.459459,0.958333,1.184211


In [126]:
percentis = resultados_time_DF_DESC.loc[["10%","20%","30%","40%","50%","60%","70%","80%","90%"]]

In [127]:
percentis

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
10%,45.6,0.458947,0.141520,0.315205,1.154737,0.797895,0.991579
20%,76.0,0.476023,0.210526,0.345614,1.220760,0.842105,1.040659
30%,132.0,0.501818,0.236577,0.377193,1.321558,0.875676,1.130556
40%,182.0,0.524912,0.269481,0.400333,1.421053,0.914765,1.167345
50%,190.0,0.551515,0.286036,0.407018,1.459459,0.958333,1.184211
60%,235.2,0.571073,0.311884,0.427306,1.500599,0.987371,1.221148
70%,296.0,0.593526,0.351968,0.472523,1.553397,1.011976,1.289474
80%,334.0,0.631997,0.372455,0.498544,1.615078,1.056188,1.337190
90%,334.0,0.650062,0.383234,0.513772,1.676647,1.106587,1.379641


In [128]:
def get_stars(array,value):
    print ()
    idx = (np.abs(array-value)).argmin()
    near = array[idx][0]
    index = np.where(array==near)[0][0]
    
    if (value > near):
        index = index + 1  
       
    return index + 1

In [129]:
stars_DF = resultados_time_DF.copy()

In [130]:
stars_DF["aprov_total"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_total"]].values, row["aprov_total"]),axis=1)

In [131]:
stars_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_casa"]].values, row["aprov_casa"]),axis=1)

In [132]:
stars_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_fora"]].values, row["aprov_fora"]),axis=1)

In [133]:
stars_DF["media_gols_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_casa"]].values, row["media_gols_casa"]),axis=1)

In [134]:
stars_DF["media_gols_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_fora"]].values, row["media_gols_fora"]),axis=1)

In [135]:
stars_DF["media_gols"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols"]].values, row["media_gols"]),axis=1)

In [136]:
stars_DF.eval("forca_casa = (3*aprov_casa + media_gols_casa)/4", inplace=True)

In [137]:
stars_DF.eval("forca_fora = (3*aprov_fora + media_gols_fora)/4", inplace=True)

In [138]:
stars_DF.eval("forca = (3*aprov_total + 1*media_gols)/4", inplace=True)

In [139]:
stars_DF.sort_values(["forca"], ascending=[0])

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols,forca_casa,forca_fora,forca
time,,,,,,,,,,
Cruzeiro,334,8,10,10,10,10,10,8.50,10.00,10.00
São Paulo,334,9,9,10,9,10,10,9.00,9.25,10.00
Corinthians,334,10,10,10,9,9,9,9.75,9.75,9.75
Grêmio,334,10,8,10,10,8,9,10.00,8.00,9.75
Atlético-MG,334,9,9,9,10,10,10,9.25,9.25,9.25
Santos,334,10,7,9,10,9,10,10.00,7.50,9.25
Internacional,304,9,8,9,8,6,7,8.75,7.50,8.50
Fluminense,334,8,10,8,8,10,9,8.00,10.00,8.25
Palmeiras,296,7,9,8,6,9,8,6.75,9.00,8.00


In [67]:
stars_DF.to_csv(path_or_buf="stars.csv",sep=";")